<span style="font-size: 14pt">MIPT, Advanced ML, Autumn 2017</span>

<span style="font-size: 16pt"> HW #9: Recommendation System

<span style="color:blue; font-size: 12pt">Andrey Saitgalin </span>
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">andrey.saitgalin@gmail.com</span>, 

<span style="color:blue; font-size: 12pt">Alexey Dral </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'">aadral@gmail.com</span>

<h1 align="center">Organization Info</h1> 

**Дополнительный материал для выполнения дз**:
<a href= "https://github.com/ml-mipt/ml-mipt-part2/tree/master/2017/lectures">Лекции по курсу</a>

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall <номер_группы> <фамилия>``, к примеру -- ``ML2017_Aut_fall 401 ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb``, к примеру -- ``ivanov_491_task9.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2017_Aut_fall Question <Содержание вопроса>``

** Дедлайн**:
<span style="color:red; font-size: 12pt">16.12.2017 23:59 </span>

--------
- **PS1**: Мы используем автоматические фильтры, и просто не найдем ваше дз, если вы не аккуратно его подпишите.
- **PS2**: Напоминаем, что дедлайны жесткие, письма пришедшие после автоматически удаляются =( чтобы соблазна не было.

<h1 align="center">Check Questions (0%)</h1> 
Вопросы отсутствуют

<h1 align="center"> Practical tasks (100%)</h1>


# Построение content-based рекомендательной системы
## Part 1
Для каждого фильма получите описание в текстовом виде с IMDB или (и) The movie database (у них есть открытое API). https://www.themoviedb.org/?language=ru. Вывести описание фильма под id 778. Также можно использовать рецензии.

In [17]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import trange
%matplotlib inline

In [33]:
import tmdbsimple as tmdb
tmdb.API_KEY = "6610a2df2a35548473ddd5aac84404b1"

In [16]:
id_to_overview = {}
search = tmdb.Search()
response = search.movie(query='The Bourne')
for i in trange(1,1000):
    try:
        movie = tmdb.Movies(i)
        id_to_overview[i] = movie.info()['overview']
    except Exception:
        continue

100%|████████████████████████████████████████████████████████████████████████████████| 999/999 [09:11<00:00,  1.81it/s]


In [46]:
len(id_to_overview)

823

In [61]:
id_to_overview[822] = tmdb.Movies(9911).info()['overview']

In [63]:
len(id_to_overview)

823

In [64]:
id_to_overview

{2: u"Taisto Kasurinen is a Finnish coal miner whose father has just committed suicide and who is framed for a crime he did not commit. In jail, he starts to dream about leaving the country and starting a new life. He escapes from prison but things don't go as planned...",
 3: u'An episode in the life of Nikander, a garbage man, involving the death of a co-worker, an affair and much more.',
 5: u"It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another.",
 8: u'Timo Novotny labels his new project an experimental music documentary film, in a remix of the celebrated film Megacities (1997), a visually refined essay on the hidden faces of several world "megacities" by leading Austrian documentarist Michael Glawogger. Novotny complements 30 % of material taken straight from the film (and re-edited) with 70 % as ye

In [66]:
id_to_number = {k: i for i, (k, _) in enumerate(id_to_overview.items())}

In [67]:
search = tmdb.Search()
response = search.movie(query='The Bourne')
print(id_to_overview.values())

[u"Taisto Kasurinen is a Finnish coal miner whose father has just committed suicide and who is framed for a crime he did not commit. In jail, he starts to dream about leaving the country and starting a new life. He escapes from prison but things don't go as planned...", u'An episode in the life of Nikander, a garbage man, involving the death of a co-worker, an affair and much more.', u"It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another.", u'Timo Novotny labels his new project an experimental music documentary film, in a remix of the celebrated film Megacities (1997), a visually refined essay on the hidden faces of several world "megacities" by leading Austrian documentarist Michael Glawogger. Novotny complements 30 % of material taken straight from the film (and re-edited) with 70 % as yet unseen footag

## Part 2 
Извлеките текстовые признаки у фильмов: tf idf, биграммы (может какие-то еще). Биграммы - статистика по биграммам, сколько какая биграмма встречается. Формулы для TF-IDF смотреть тут: https://en.wikipedia.org/wiki/Tf%E2%80%93idf 
    1. Используйте TF с double 0.5 normalization
    2. Используйте стандартный IDF: 
$$idf(t, D) = \frac{log N}{(1 + |d in D : t in d|)}$$

In [68]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
bigram_vectorizer_for_tf_idf = CountVectorizer(ngram_range=(1, 1))
X = bigram_vectorizer.fit_transform(list(id_to_overview.values()))
X_tf_idf = bigram_vectorizer_for_tf_idf.fit_transform(list(id_to_overview.values()))

In [69]:
print(X_tf_idf[id_to_number[778], 778])

0


In [70]:
print(X)

  (0, 3121)	1
  (0, 13132)	1
  (0, 9292)	1
  (0, 31946)	1
  (0, 5410)	1
  (0, 24791)	1
  (0, 12475)	1
  (0, 10359)	1
  (0, 14022)	1
  (0, 18800)	1
  (0, 21595)	1
  (0, 28817)	1
  (0, 2357)	1
  (0, 7495)	1
  (0, 30517)	1
  (0, 18597)	1
  (0, 409)	1
  (0, 9453)	1
  (0, 32488)	1
  (0, 28828)	1
  (0, 14122)	1
  (0, 17444)	1
  (0, 16064)	1
  (0, 6968)	1
  (0, 21841)	1
  :	:
  (822, 14692)	1
  (822, 33916)	1
  (822, 29778)	1
  (822, 24200)	1
  (822, 24281)	1
  (822, 863)	1
  (822, 35459)	1
  (822, 4013)	1
  (822, 919)	1
  (822, 21053)	1
  (822, 969)	1
  (822, 31085)	1
  (822, 23928)	2
  (822, 5652)	1
  (822, 12920)	1
  (822, 5447)	1
  (822, 14767)	1
  (822, 827)	1
  (822, 14632)	1
  (822, 22089)	2
  (822, 30255)	4
  (822, 32348)	2
  (822, 15911)	1
  (822, 7683)	1
  (822, 1720)	1


In [80]:
def tf(t,d):
    return 0.5 + 0.5 * X_tf_idf[id_to_number[d], t] / max(X_tf_idf[id_to_number[d], :].data)

In [73]:
def idf(t,D):
    return np.log(len(D) / len(X_tf_idf[:, t].indices))

In [74]:
def tf_idf(t,d,D):
    return tf(t,d) * idf(t,D)

In [75]:
from scipy.sparse import csr_matrix

In [76]:
number_to_id = np.sort(list(id_to_overview.keys()))

#csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
#where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k].

data = []
row_ind = []
col_ind = []

for d in number_to_id:
    for t in X_tf_idf[id_to_number[d], :].indices:
        data.append(tf_idf(t, d, number_to_id))
        row_ind.append(id_to_number[d])
        col_ind.append(t)

tf_idf_matrix = csr_matrix((data, (row_ind, col_ind)))

In [77]:
print(tf_idf_matrix)

  (0, 156)	1.2972734327
  (0, 427)	0.0
  (0, 562)	0.732408192445
  (0, 1133)	1.07295860829
  (0, 1476)	4.47530413378
  (0, 1554)	3.54867331943
  (0, 1557)	3.74208540426
  (0, 1734)	2.47571471114
  (0, 1778)	2.44237443075
  (0, 2099)	3.74208540426
  (0, 2241)	3.00723967101
  (0, 2288)	3.08331520886
  (0, 2571)	2.93781283151
  (0, 2778)	2.06069496891
  (0, 2867)	4.01239547633
  (0, 2941)	0.732408192445
  (0, 2982)	3.74208540426
  (0, 3028)	0.732408192445
  (0, 3180)	2.47571471114
  (0, 3384)	1.19450631282
  (0, 3403)	1.09861228867
  (0, 3725)	0.0
  (0, 3915)	0.577622650467
  (0, 3942)	3.00723967101
  (0, 4058)	2.24486388666
  :	:
  (822, 3746)	4.19559762542
  (822, 3831)	3.99225748435
  (822, 4520)	3.50820506649
  (822, 4621)	3.18741651739
  (822, 4894)	3.07498807864
  (822, 4908)	2.05989804125
  (822, 5146)	0.0
  (822, 5339)	3.32688123696
  (822, 5399)	3.46872960996
  (822, 5493)	2.28972602883
  (822, 5509)	2.63719231574
  (822, 5547)	3.76162075906
  (822, 5714)	4.19559762542
  (822, 57

## Part 3 
Выведите подсчитанные признаки из пункта 2 для фильма с id 99114.

не существует фильма с таким id. Поэтому сделаем для фильма с id=9911

In [79]:
print(X[id_to_number[822], :].data)
print(tf_idf_matrix[id_to_number[822], :].data)
print(id_to_overview[822])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 1 1 1 1 1 1]
[ 0.          0.82395922  3.82489982  1.09861229  2.35162066  2.38354037
  3.99225748  3.10735104  3.38314463  2.78517905  2.59930193  0.
  3.68998569  3.16463078  0.          2.82090009  3.82489982  2.24679921
  2.31828184  3.82489982  3.22804882  3.04533226  3.04533226  0.
  1.60943791  1.45943261  0.          2.55089804  0.82395922  4.20984608]
A group of people who feel betrayed by their government and let down by their police force form a modern-day outlaw posse in order to right what they see as the wrongs of society.


## Part 4
Для каждого пользователя найдите топ жанров, которые нравятся пользователю (можно делать как на семинаре top 5, top 3 и top 2) и порекомендуйте таким пользователя фильмы близкие по текстовому описанию (см. пункт 2). Для оценки результатов пункта 3 используйте метрику HitRate и метрики ранжирования (N)DCG. Формулу для подсчета HitRate можно взять на семинаре,  (N)DCG: https://en.wikipedia.org/wiki/Discounted_cumulative_gain



In [ ]:
# put your code here